# Data preparation - Seatle AirBnB

In [84]:
#Importing standard liabraries

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns

In [85]:
# Uploading AirBnB data sets

calendar_df = pd.read_csv('calendar.csv')

listing_df = pd.read_csv('listings.csv')

reviews_df = pd.read_csv('reviews.csv')

As we've seen in the __Data understanding__ step: 

- Some of the attributes from Seattle dataset have missing data
    - Luckily for us only a small proportion of those attributes we are going to use in the model have around 10% of missing values. The rest of data is complete.
- We've identified highly correlated attributes from the same categories
    - Those attributes will be excluded from the training model 

It certainly does not make sense to use all categorical and numerical values for our data model. Hence we will focus on the most reasonable predictors. For instance for numerical variables we will exclude all highly  correlated attributes, this will save us from multicollinearity issues in the future. (See Data Understanding - correlation heatmap)

As for categorical, we will have to apply a bit of common sense and exclude such as: state, city, picture_url, name etc.

In [86]:
# Removing all rows from listing_df that don't have price records

listing_df = listing_df.dropna(subset=['price'],axis=0)


In [87]:
# Cleaning categorical variables. 



listings_clean_df = listing_df.drop(columns = ['amenities','calendar_updated','cancellation_policy','city',
                                               'space', 'neighborhood_overview', 'notes',
                                               'medium_url', 'xl_picture_url', 'host_url', 'is_location_exact',
                                               'country','country_code','description','experiences_offered','calendar_last_scraped',
                                               'host_has_profile_pic','host_id','host_name','host_picture_url', 'host_about', 'host_verifications',
                                               'host_thumbnail_url','listing_url','market','name','neighbourhood_group_cleansed',
                                               'jurisdiction_names', 'require_guest_profile_picture', 'smart_location',
                                               'picture_url','scrape_id','state','street','summary','thumbnail_url','zipcode'])



listings_clean_df.columns

Index(['id', 'last_scraped', 'transit', 'host_since', 'host_location',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'square_feet', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_s

In [88]:
# Cleaning categorical variables. 

listings_clean_df = listings_clean_df.drop(columns = ['id','host_response_rate','accommodates','bedrooms',
                                                      'host_listings_count','host_total_listings_count', 
                                                     'guests_included','availability_60','availability_90', 'availability_365',
                                                     'review_scores_accuracy','review_scores_cleanliness','review_scores_checkin',
                                                     'review_scores_communication','review_scores_value'])
listings_clean_df.columns

Index(['last_scraped', 'transit', 'host_since', 'host_location',
       'host_response_time', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'bathrooms', 'beds', 'bed_type', 'square_feet', 'price',
       'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee',
       'extra_people', 'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'number_of_reviews', 'first_review', 'last_review',
       'review_scores_rating', 'review_scores_location', 'requires_license',
       'license', 'instant_bookable', 'require_guest_phone_verification',
       'calculated_host_listings_count', 'reviews_per_month'],
      dtype='object')

In [89]:
# Let's review attributes with more than 10% of missing values 

missing_10_pct_values = set(listings_clean_df.columns[listings_clean_df.isnull().mean() > 0.3])

missing_10_pct_values

{'license', 'monthly_price', 'security_deposit', 'square_feet', 'weekly_price'}

In [90]:
# Let's drop columns with more than 10% missing values

listings_clean_df = listings_clean_df.drop(columns = ['first_review','host_acceptance_rate',
                                                     'last_review','license','monthly_price','neighbourhood',
                                                     'reviews_per_month','security_deposit','square_feet',
                                                     'transit','weekly_price'])

listings_clean_df.head(5)

,last_scraped,host_since,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,latitude,longitude,...,maximum_nights,has_availability,availability_30,number_of_reviews,review_scores_rating,review_scores_location,requires_license,instant_bookable,require_guest_phone_verification,calculated_host_listings_count
0,2016-01-04,2011-08-11,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.636289,-122.371025,...,365,t,14,207,95.0,9.0,f,f,f,2
1,2016-01-04,2013-02-21,"Seattle, Washington, United States",within an hour,t,Queen Anne,t,West Queen Anne,47.639123,-122.365666,...,90,t,13,43,96.0,10.0,f,f,t,6
2,2016-01-04,2014-06-12,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.629724,-122.369483,...,30,t,1,20,97.0,10.0,f,f,f,2
3,2016-01-04,2013-11-06,"Seattle, Washington, United States",NaN,f,Queen Anne,t,West Queen Anne,47.638473,-122.369279,...,1125,t,0,0,NaN,NaN,f,f,f,1
4,2016-01-04,2011-11-29,"Seattle, Washington, United States",within an hour,f,Queen Anne,t,West Queen Anne,47.632918,-122.372471,...,1125,t,30,38,92.0,9.0,f,f,f,1


We can merge two columns last_scraped and host_since to calculate for how long host was on the market

In [91]:
import datetime

x = str('2015-02-24')

x = datetime.datetime.strptime(x, "%Y-%m-%d").date()

x

datetime.date(2015, 2, 24)

In [92]:
# function for converting string to datetime

def str_to_date(x):
        x = datetime.datetime.strptime(x, "%Y-%m-%d").date()
        return x

In [93]:
# converting last_scraped into datetime object

listings_clean_df['last_scraped'] = listings_clean_df['last_scraped'].apply(str_to_date)

In [94]:
# converting last_scraped into datetime object
# we fill nan for host_since with '2016-01-04' - the scrape date

listings_clean_df['host_since'] = listings_clean_df['host_since'].fillna(str('2016-01-04'))

listings_clean_df['host_since'] = listings_clean_df['host_since'].astype(str).apply(str_to_date)

In [95]:
# Calculating days with abnb

listings_clean_df['days_with_abnb'] = listings_clean_df['last_scraped'] - listings_clean_df['host_since']

listings_clean_df['days_with_abnb'] = listings_clean_df['days_with_abnb'] / np.timedelta64(1, 'D')

In [96]:
listings_clean_df.columns

Index(['last_scraped', 'host_since', 'host_location', 'host_response_time',
       'host_is_superhost', 'host_neighbourhood', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'bathrooms', 'beds', 'bed_type', 'price', 'cleaning_fee',
       'extra_people', 'minimum_nights', 'maximum_nights', 'has_availability',
       'availability_30', 'number_of_reviews', 'review_scores_rating',
       'review_scores_location', 'requires_license', 'instant_bookable',
       'require_guest_phone_verification', 'calculated_host_listings_count',
       'days_with_abnb'],
      dtype='object')

In [97]:
listings_clean_df.head(5)

,last_scraped,host_since,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,latitude,longitude,...,has_availability,availability_30,number_of_reviews,review_scores_rating,review_scores_location,requires_license,instant_bookable,require_guest_phone_verification,calculated_host_listings_count,days_with_abnb
0,2016-01-04,2011-08-11,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.636289,-122.371025,...,t,14,207,95.0,9.0,f,f,f,2,1607.0
1,2016-01-04,2013-02-21,"Seattle, Washington, United States",within an hour,t,Queen Anne,t,West Queen Anne,47.639123,-122.365666,...,t,13,43,96.0,10.0,f,f,t,6,1047.0
2,2016-01-04,2014-06-12,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.629724,-122.369483,...,t,1,20,97.0,10.0,f,f,f,2,571.0
3,2016-01-04,2013-11-06,"Seattle, Washington, United States",NaN,f,Queen Anne,t,West Queen Anne,47.638473,-122.369279,...,t,0,0,NaN,NaN,f,f,f,1,789.0
4,2016-01-04,2011-11-29,"Seattle, Washington, United States",within an hour,f,Queen Anne,t,West Queen Anne,47.632918,-122.372471,...,t,30,38,92.0,9.0,f,f,f,1,1497.0


In [98]:
# Now we can drop last_scraped & 
listings_clean_df = listings_clean_df.drop(columns = ['last_scraped','host_since'])

listings_clean_df.head(5)

,host_location,host_response_time,host_is_superhost,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,...,has_availability,availability_30,number_of_reviews,review_scores_rating,review_scores_location,requires_license,instant_bookable,require_guest_phone_verification,calculated_host_listings_count,days_with_abnb
0,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.636289,-122.371025,Apartment,Entire home/apt,...,t,14,207,95.0,9.0,f,f,f,2,1607.0
1,"Seattle, Washington, United States",within an hour,t,Queen Anne,t,West Queen Anne,47.639123,-122.365666,Apartment,Entire home/apt,...,t,13,43,96.0,10.0,f,f,t,6,1047.0
2,"Seattle, Washington, United States",within a few hours,f,Queen Anne,t,West Queen Anne,47.629724,-122.369483,House,Entire home/apt,...,t,1,20,97.0,10.0,f,f,f,2,571.0
3,"Seattle, Washington, United States",NaN,f,Queen Anne,t,West Queen Anne,47.638473,-122.369279,Apartment,Entire home/apt,...,t,0,0,NaN,NaN,f,f,f,1,789.0
4,"Seattle, Washington, United States",within an hour,f,Queen Anne,t,West Queen Anne,47.632918,-122.372471,House,Entire home/apt,...,t,30,38,92.0,9.0,f,f,f,1,1497.0


In [99]:
listings_clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3818 entries, 0 to 3817
Data columns (total 28 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   host_location                     3810 non-null   object 
 1   host_response_time                3295 non-null   object 
 2   host_is_superhost                 3816 non-null   object 
 3   host_neighbourhood                3518 non-null   object 
 4   host_identity_verified            3816 non-null   object 
 5   neighbourhood_cleansed            3818 non-null   object 
 6   latitude                          3818 non-null   float64
 7   longitude                         3818 non-null   float64
 8   property_type                     3817 non-null   object 
 9   room_type                         3818 non-null   object 
 10  bathrooms                         3802 non-null   float64
 11  beds                              3817 non-null   float64
 12  bed_ty

## Data Cleaning

### Cleaning numerical values

In [100]:
# Function for filling missing values with mean - this will work on 

fill_mean = lambda col: col.fillna(col.mean())


In [101]:
# Now let's see what are the columns that have missing values

listings_clean_df.columns[listings_clean_df.isna().any()].tolist()

['host_location',
 'host_response_time',
 'host_is_superhost',
 'host_neighbourhood',
 'host_identity_verified',
 'property_type',
 'bathrooms',
 'beds',
 'cleaning_fee',
 'review_scores_rating',
 'review_scores_location']

In [102]:
# Let's re-use our financial_to_float function from data exploration step

def financial_to_float(x):
        x = x.replace(",", "")
        return float(x.strip('$'))

In [103]:
financials = ['price','cleaning_fee','extra_people']

for i in financials:
    listings_clean_df[i] = listing_df[i].fillna('$0').apply(financial_to_float)

In [104]:
# Filling missing numeric columns with mean values  

numerics = [['bathrooms','beds','cleaning_fee','review_scores_rating','review_scores_location','extra_people']]

for i in numerics:
    listings_clean_df[i] = listings_clean_df[i].apply(fill_mean, axis=0)
    

In [105]:
listings_clean_df['bathrooms'] = listings_clean_df[['bathrooms']].apply(fill_mean, axis=0)

In [106]:
listings_clean_df[['bathrooms','beds','cleaning_fee','review_scores_rating','review_scores_location']].head(5)

,bathrooms,beds,cleaning_fee,review_scores_rating,review_scores_location
0,1.0,1.0,0.0,95.000000,9.000000
1,1.0,1.0,40.0,96.000000,10.000000
2,4.5,7.0,300.0,97.000000,10.000000
3,1.0,2.0,0.0,94.539262,9.608916
4,2.0,3.0,125.0,92.000000,9.000000


### Cleaning categorical variables

In [107]:
listings_clean_df = listings_clean_df.drop(['host_location','host_neighbourhood'], axis = 1)

In [108]:
# Here we will replace missing categorical variables with the most frequent value - mode 
# Let's create a function for that

def impute_nan_most_frequent_category(DataFrame,ColName):
    # .mode()[0] - gives first category name
     most_frequent_category=DataFrame[ColName].mode()[0]
    
    # replace nan values with most occured category
     DataFrame[ColName + "_Imputed"] = DataFrame[ColName]
     DataFrame[ColName + "_Imputed"].fillna(most_frequent_category,inplace=True)

In [109]:
categorical_varaibles = ['host_response_time','host_is_superhost','host_identity_verified','property_type']

for i in categorical_varaibles:
    impute_nan_most_frequent_category(listings_clean_df,i)
    

In [110]:
listings_clean_df.head(5)

,host_response_time,host_is_superhost,host_identity_verified,neighbourhood_cleansed,latitude,longitude,property_type,room_type,bathrooms,beds,...,review_scores_location,requires_license,instant_bookable,require_guest_phone_verification,calculated_host_listings_count,days_with_abnb,host_response_time_Imputed,host_is_superhost_Imputed,host_identity_verified_Imputed,property_type_Imputed
0,within a few hours,f,t,West Queen Anne,47.636289,-122.371025,Apartment,Entire home/apt,1.0,1.0,...,9.000000,f,f,f,2,1607.0,within a few hours,f,t,Apartment
1,within an hour,t,t,West Queen Anne,47.639123,-122.365666,Apartment,Entire home/apt,1.0,1.0,...,10.000000,f,f,t,6,1047.0,within an hour,t,t,Apartment
2,within a few hours,f,t,West Queen Anne,47.629724,-122.369483,House,Entire home/apt,4.5,7.0,...,10.000000,f,f,f,2,571.0,within a few hours,f,t,House
3,NaN,f,t,West Queen Anne,47.638473,-122.369279,Apartment,Entire home/apt,1.0,2.0,...,9.608916,f,f,f,1,789.0,within an hour,f,t,Apartment
4,within an hour,f,t,West Queen Anne,47.632918,-122.372471,House,Entire home/apt,2.0,3.0,...,9.000000,f,f,f,1,1497.0,within an hour,f,t,House


In [112]:
listings_clean_df = listings_clean_df.drop(columns = ['host_response_time','host_is_superhost','host_identity_verified','property_type'])

In [113]:
listings_clean_df.columns.to_list()

['neighbourhood_cleansed',
 'latitude',
 'longitude',
 'room_type',
 'bathrooms',
 'beds',
 'bed_type',
 'price',
 'cleaning_fee',
 'extra_people',
 'minimum_nights',
 'maximum_nights',
 'has_availability',
 'availability_30',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_location',
 'requires_license',
 'instant_bookable',
 'require_guest_phone_verification',
 'calculated_host_listings_count',
 'days_with_abnb',
 'host_response_time_Imputed',
 'host_is_superhost_Imputed',
 'host_identity_verified_Imputed',
 'property_type_Imputed']

In [114]:
# Dummy the categorical variables
cat_vars = listings_clean_df.select_dtypes(include=['object']).copy().columns

for var in  cat_vars:
        # for each cat add dummy var, drop original column
        listings_clean_df = pd.concat([listings_clean_df.drop(var, axis=1), pd.get_dummies(listings_clean_df[var], prefix=var, prefix_sep='_d', drop_first=True)], axis=1)

In [115]:
listings_clean_df.head(5)

,latitude,longitude,bathrooms,beds,price,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_30,...,property_type_Imputed_dChalet,property_type_Imputed_dCondominium,property_type_Imputed_dDorm,property_type_Imputed_dHouse,property_type_Imputed_dLoft,property_type_Imputed_dOther,property_type_Imputed_dTent,property_type_Imputed_dTownhouse,property_type_Imputed_dTreehouse,property_type_Imputed_dYurt
0,47.636289,-122.371025,1.0,1.0,85.0,0.0,5.0,1,365,14,...,0,0,0,0,0,0,0,0,0,0
1,47.639123,-122.365666,1.0,1.0,150.0,40.0,0.0,2,90,13,...,0,0,0,0,0,0,0,0,0,0
2,47.629724,-122.369483,4.5,7.0,975.0,300.0,25.0,4,30,1,...,0,0,0,1,0,0,0,0,0,0
3,47.638473,-122.369279,1.0,2.0,100.0,0.0,0.0,1,1125,0,...,0,0,0,0,0,0,0,0,0,0
4,47.632918,-122.372471,2.0,3.0,450.0,125.0,15.0,1,1125,30,...,0,0,0,1,0,0,0,0,0,0


In [118]:
print(listings_clean_df.info())

listings_clean_df.columns.to_list()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3818 entries, 0 to 3817
Columns: 129 entries, latitude to property_type_Imputed_dYurt
dtypes: float64(10), int64(5), uint8(114)
memory usage: 902.3 KB
None


['latitude',
 'longitude',
 'bathrooms',
 'beds',
 'price',
 'cleaning_fee',
 'extra_people',
 'minimum_nights',
 'maximum_nights',
 'availability_30',
 'number_of_reviews',
 'review_scores_rating',
 'review_scores_location',
 'calculated_host_listings_count',
 'days_with_abnb',
 'neighbourhood_cleansed_dAlki',
 'neighbourhood_cleansed_dArbor Heights',
 'neighbourhood_cleansed_dAtlantic',
 'neighbourhood_cleansed_dBelltown',
 'neighbourhood_cleansed_dBitter Lake',
 'neighbourhood_cleansed_dBriarcliff',
 'neighbourhood_cleansed_dBrighton',
 'neighbourhood_cleansed_dBroadview',
 'neighbourhood_cleansed_dBroadway',
 'neighbourhood_cleansed_dBryant',
 'neighbourhood_cleansed_dCedar Park',
 'neighbourhood_cleansed_dCentral Business District',
 'neighbourhood_cleansed_dColumbia City',
 'neighbourhood_cleansed_dCrown Hill',
 'neighbourhood_cleansed_dDunlap',
 'neighbourhood_cleansed_dEast Queen Anne',
 'neighbourhood_cleansed_dEastlake',
 'neighbourhood_cleansed_dFairmount Park',
 'neighbourh

In [119]:
listings_clean_df.to_csv('listings_clean.csv',index=False)